In [1]:
import os

In [2]:
%pwd

'c:\\projects\\Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\projects\\Text-Summarizer'

In [5]:
#update entity it is nothing but return type of function
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
#4th step
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([Path(str(self.config.artifacts_root))])



        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([Path(str(config.root_dir))])


        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(str(config.root_dir)),
            source_URL=config.source_URL,
            local_data_file=Path(str(config.local_data_file)),
            unzip_dir=Path(str(config.unzip_dir)),
        )


        return data_ingestion_config
    
        

In [8]:
#update my components
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self) :
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file,
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self) :
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)
       
        

In [10]:
#update the pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception:
    raise


[2026-01-07 02:30:31,698: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-07 02:30:31,705: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-07 02:30:31,708: INFO: common: created directory at: artifacts]
[2026-01-07 02:30:31,709: INFO: common: created directory at: artifacts\data_ingestion]
[2026-01-07 02:30:31,713: INFO: 231812062: File already exists of size: ~7718.35 KB]
